# Importing Software

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, GroupKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVR
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.linear_model import LassoCV, Lasso, LassoLarsIC, ElasticNet, LassoLarsCV
from sklearn.externals import joblib
from scipy.stats import randint as sp_randint
from matplotlib import pyplot as plt
from sklearn.feature_selection import RFECV

/Users/ADB0097/miniconda3/envs/ml3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
%matplotlib inline
import biom
import calour as ca
from scipy import stats
import scipy
import pickle
import time

# Importing Data

In [17]:
#igg_data = ca.read_amplicon('feature-table.biom', 'igg-shelflife-metadata.tsv', min_reads=0, normalize= 1000)
igg_data = ca.read_amplicon('feature-table-nochlomito.biom', 'igg-shelflife-metadata.tsv', min_reads=0, normalize= 1000)

2025-07-16 14:30:37 WARNING These have metadata but do not have data - dropped (10): {'d8_3', 'd4_3', 'd6_1', 'd7_3', 'd9_2', 'd2_3', 'd2_2', 'd12_3', 'd8_2', 'd6_2'}
2025-07-16 14:30:37 WARNING Do you forget to normalize your data? It is required before running this function


# Model Building

## CFU counts

In [19]:
igg_data.sample_metadata.lot.value_counts()
#springPMI = springPMI.filter_samples('control', 'n')
#springPMI.sample_metadata.host_subject_id.value_counts()

1    14
2    11
3    10
Name: lot, dtype: int64

In [21]:
gkf = GroupKFold(3)

X = igg_data.data
y = igg_data.sample_metadata['logcfu']
groups = igg_data.sample_metadata['lot']
param_grid = {"max_depth": [3, None],
          "max_features": [3, 10],
          "min_samples_split": [ 3, 10],
          "min_samples_leaf": [1, 3, 10],
          "bootstrap": [True, False]}
rf = RandomForestRegressor(n_estimators=100, random_state=999)
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=4)
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f9ed29fb3d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n...mators=100, n_jobs=None,
                                             oob_score=False, random_state=999,
                                             ver

In [22]:
gs.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__bootstrap', 'estimator__ccp_alpha', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__max_samples', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [23]:
print('The best error rate is', -gs.best_score_)

The best error rate is 1.099632184574018


In [24]:
rfecv = RFECV(estimator=rf, step=1, cv=list(gkf.split(X, y, groups)))
rfecv.fit(X, y)

RFECV(cv=[(array([ 1,  2,  4,  5,  7,  8, 10, 11, 13, 15, 16, 18, 19, 22, 23, 25, 27,
       28, 29, 31, 34]),
           array([ 0,  3,  6,  9, 12, 14, 17, 20, 21, 24, 26, 30, 32, 33])),
          (array([ 0,  2,  3,  5,  6,  8,  9, 11, 12, 14, 16, 17, 19, 20, 21, 23, 24,
       26, 28, 29, 30, 32, 33, 34]),
           array([ 1,  4,  7, 10, 13, 15, 18, 22, 25, 27, 31])),
          (array([ 0,  1,  3,  4,  6,  7,  9, 10, 12, 13, 14, 15, 17, 18, 20, 21, 22,
       24, 25, 26, 27, 30, 31, 32, 33]),
           array([ 2,  5,  8, 11, 16, 19, 23, 28, 29, 3...
                                      criterion='mse', max_depth=None,
                                      max_features='auto', max_leaf_nodes=None,
                                      max_samples=None,
                                      min_impurity_decrease=0.0,
                                      min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                        

In [25]:
#print('The best error rate is', -rfecv.s)
print("Optimal number of features : %d" % rfecv.n_features_)

Optimal number of features : 15


In [26]:
print('The best error rate is', -gs.best_score_)

The best error rate is 1.099632184574018


In [27]:
joblib.dump(gs.best_estimator_, 'igg_data_cfu.pkl')

['igg_data_cfu.pkl']

In [28]:
model_igg_data_cfu = joblib.load('igg_data_cfu.pkl')

In [29]:
model_igg_data_cfu

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

In [30]:
importances = model_igg_data_cfu.feature_importances_
std = np.std([tree.feature_importances_ for tree in model_igg_data_cfu.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

#for f in range(X_CMU.shape[1]):
#    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
feature_metadata = igg_data.feature_metadata
np.savetxt("feature_metadata.csv", feature_metadata, delimiter=",", fmt='%s')


cfu_importances = []
for f in range(X.shape[1]):
    cfu_importances += (indices[f], importances[indices[f]])
print(importances)


Feature ranking:
[0.00000000e+00 1.96050784e-03 1.89939306e-03 0.00000000e+00
 7.10299402e-04 7.59403376e-04 9.38499886e-03 1.92304818e-03
 2.16623849e-04 0.00000000e+00 8.27400120e-03 1.08345934e-02
 2.61892539e-03 0.00000000e+00 2.15374633e-03 1.11363858e-03
 0.00000000e+00 3.13102636e-05 0.00000000e+00 8.97611474e-05
 2.90054365e-05 4.33901176e-06 2.12044480e-03 0.00000000e+00
 1.91709406e-03 1.52012050e-04 1.90706769e-03 0.00000000e+00
 0.00000000e+00 7.78013662e-04 0.00000000e+00 2.72231175e-05
 0.00000000e+00 1.24100032e-04 5.03265174e-03 0.00000000e+00
 5.26344682e-04 1.20490449e-04 0.00000000e+00 8.54100251e-03
 3.23337628e-03 2.14215337e-04 3.35373162e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.23438412e-05 4.61545284e-05
 1.93928292e-03 0.00000000e+00 0.00000000e+00 7.92261708e-04
 0.00000000e+00 2.39668464e-05 7.51295597e-04 2.92866303e-02
 4.30218277e-06 1.30928063e-03 1.27569908e-03 1.47768277e-03
 0.00000000e+00 1.12403711e-03 2.04204799e-04 9.11997481e-04
 1.0624

In [ ]:
for f in range(X.shape[1]):
    print(igg_data.feature_metadata['_feature_id'][f], importances[indices[f]])

## TBARS

In [32]:
igg_data.sample_metadata.lot.value_counts()

1    14
2    11
3    10
Name: lot, dtype: int64

In [33]:
gkf = GroupKFold(3)

X = igg_data.data
y = igg_data.sample_metadata['avgTBARS']
groups = igg_data.sample_metadata['lot']
param_grid = {"max_depth": [3, None],
          "max_features": [3, 10],
          "min_samples_split": [ 3, 10],
          "min_samples_leaf": [1, 3, 10],
          "bootstrap": [True, False]}
rf = RandomForestRegressor(n_estimators=100, random_state=999)
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=4)
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f9ee06773d0>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n...mators=100, n_jobs=None,
                                             oob_score=False, random_state=999,
                                             ver

In [34]:
gs.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__bootstrap', 'estimator__ccp_alpha', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__max_samples', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [35]:
print('The best error rate is', -gs.best_score_)

The best error rate is 2.976100444285714


In [36]:
rfecv = RFECV(estimator=rf, step=1, cv=list(gkf.split(X, y, groups)))
rfecv.fit(X, y)

RFECV(cv=[(array([ 1,  2,  4,  5,  7,  8, 10, 11, 13, 15, 16, 18, 19, 22, 23, 25, 27,
       28, 29, 31, 34]),
           array([ 0,  3,  6,  9, 12, 14, 17, 20, 21, 24, 26, 30, 32, 33])),
          (array([ 0,  2,  3,  5,  6,  8,  9, 11, 12, 14, 16, 17, 19, 20, 21, 23, 24,
       26, 28, 29, 30, 32, 33, 34]),
           array([ 1,  4,  7, 10, 13, 15, 18, 22, 25, 27, 31])),
          (array([ 0,  1,  3,  4,  6,  7,  9, 10, 12, 13, 14, 15, 17, 18, 20, 21, 22,
       24, 25, 26, 27, 30, 31, 32, 33]),
           array([ 2,  5,  8, 11, 16, 19, 23, 28, 29, 3...
                                      criterion='mse', max_depth=None,
                                      max_features='auto', max_leaf_nodes=None,
                                      max_samples=None,
                                      min_impurity_decrease=0.0,
                                      min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                        

In [37]:
#print('The best error rate is', -rfecv.s)
print("Optimal number of features : %d" % rfecv.n_features_)

Optimal number of features : 6


In [38]:
print('The best error rate is', -gs.best_score_)

The best error rate is 2.976100444285714


In [39]:
joblib.dump(gs.best_estimator_, 'igg_data_tbars.pkl')

['igg_data_tbars.pkl']

In [40]:
model_igg_data_tbars = joblib.load('igg_data_tbars.pkl')

In [41]:
model_igg_data_tbars

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

In [42]:
importances = model_igg_data_tbars.feature_importances_
std = np.std([tree.feature_importances_ for tree in model_igg_data_tbars.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

feature_metadata = igg_data.feature_metadata
np.savetxt("feature_metadata.csv", feature_metadata, delimiter=",", fmt='%s')


tbars_importances = []
for f in range(X.shape[1]):
    tbars_importances += (indices[f], importances[indices[f]])
print(importances)


Feature ranking:
[2.57071061e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.09939634e-04 1.42251313e-02 2.97113473e-04
 0.00000000e+00 0.00000000e+00 1.05969833e-02 1.17263058e-02
 2.51392233e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.07694115e-04 0.00000000e+00 1.40361667e-04
 3.21319716e-03 2.38481000e-05 4.25720757e-04 0.00000000e+00
 0.00000000e+00 2.09842149e-05 0.00000000e+00 0.00000000e+00
 5.11723788e-04 1.05960407e-04 0.00000000e+00 4.55390382e-06
 0.00000000e+00 1.98668739e-04 1.10691378e-03 4.78235846e-04
 1.37673352e-04 3.11556250e-04 0.00000000e+00 8.16071192e-03
 2.87211534e-04 6.50406639e-04 3.47415496e-04 6.70956171e-08
 9.89851467e-07 0.00000000e+00 1.13719108e-05 2.16758310e-03
 0.00000000e+00 2.11833639e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.63837271e-03 3.52521926e-04 3.86065029e-02
 2.15237878e-04 7.38069957e-04 8.25692280e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 2.21373688e-04 4.00591397e-03
 4.5917

In [43]:
for f in range(X.shape[1]):
    print(igg_data.feature_metadata['_feature_id'][f], importances[indices[f]])

46d3ba234416dee513f80752246a2066 0.06227073462515
a781986c6888ab56b7925420d1e54abe 0.047452023197532334
2f2861bb61bd6c8cbd0d4163ec7bba53 0.0452282676451122
4325e118c9a634863c3a0977a0307615 0.04501788543990529
01d69d0ad44bd1fc99e0c4c2a0e8b313 0.04090227850342546
f42044fb1499aab36b0f3a0675039af2 0.03860650291127079
a45157ca6c33a380b00afed7bfb8b3d6 0.03824277044058571
6f0b872f7034f80cdf0f2fc4932697be 0.03785904848874096
15999689585a670d17275ebd210b39ff 0.03630358381067845
8c5c984f156f3e2c787f008ffcf69613 0.033282378468489825
8dbeabfbf721f4b9ee384b433f2ed270 0.021427439155010455
62f9ed0cc77aa2ad42da339e47297b6a 0.016746456264411843
e9fabaf81fba070afe35405e72b00992 0.01599450405002911
94e0875524c2cbde20c27b92c82a381b 0.015214073595694015
e9c9ea6a772243fadbcda9387790c0ca 0.01422513126708487
aeb2228a033d1764bca1b7eb507af64f 0.012803576037389908
889ec2416a0f378d5bd89b2857ba29f5 0.01172630579794122
9197cc47bf661baac6a81e6369744b9b 0.010826977650118261
74b459a38e8b6ae52e5586f7c94c4828 0.01074278

## Days

In [44]:
gkf = GroupKFold(3)

X = igg_data.data
y = igg_data.sample_metadata['day']
groups = igg_data.sample_metadata['lot']
param_grid = {"max_depth": [3, None],
          "max_features": [3, 10],
          "min_samples_split": [ 3, 10],
          "min_samples_leaf": [1, 3, 10],
          "bootstrap": [True, False]}
rf = RandomForestRegressor(n_estimators=100, random_state=999)
gs = GridSearchCV(rf, param_grid=param_grid, cv=gkf.split(X, y, groups), scoring='neg_mean_absolute_error', n_jobs=4)
gs.fit(X, y)

GridSearchCV(cv=<generator object _BaseKFold.split at 0x7f9f12be0d50>,
             error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n...mators=100, n_jobs=None,
                                             oob_score=False, random_state=999,
                                             ver

In [45]:
gs.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__bootstrap', 'estimator__ccp_alpha', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__max_samples', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [46]:
print('The best error rate is', -gs.best_score_)

The best error rate is 2.4835757575757573


In [47]:
rfecv = RFECV(estimator=rf, step=1, cv=list(gkf.split(X, y, groups)))
rfecv.fit(X, y)

RFECV(cv=[(array([ 1,  2,  4,  5,  7,  8, 10, 11, 13, 15, 16, 18, 19, 22, 23, 25, 27,
       28, 29, 31, 34]),
           array([ 0,  3,  6,  9, 12, 14, 17, 20, 21, 24, 26, 30, 32, 33])),
          (array([ 0,  2,  3,  5,  6,  8,  9, 11, 12, 14, 16, 17, 19, 20, 21, 23, 24,
       26, 28, 29, 30, 32, 33, 34]),
           array([ 1,  4,  7, 10, 13, 15, 18, 22, 25, 27, 31])),
          (array([ 0,  1,  3,  4,  6,  7,  9, 10, 12, 13, 14, 15, 17, 18, 20, 21, 22,
       24, 25, 26, 27, 30, 31, 32, 33]),
           array([ 2,  5,  8, 11, 16, 19, 23, 28, 29, 3...
                                      criterion='mse', max_depth=None,
                                      max_features='auto', max_leaf_nodes=None,
                                      max_samples=None,
                                      min_impurity_decrease=0.0,
                                      min_impurity_split=None,
                                      min_samples_leaf=1, min_samples_split=2,
                        

In [52]:
#print('The best error rate is', -rfecv.s)
print("Optimal number of features : %d" % rfecv.n_features_)

Optimal number of features : 9


In [53]:
print('The best error rate is', -gs.best_score_)

The best error rate is 2.4835757575757573


In [54]:
joblib.dump(gs.best_estimator_, 'igg_data_day.pkl')

['igg_data_day.pkl']

In [57]:
model_igg_data_day = joblib.load('igg_data_day.pkl')

In [58]:
model_igg_data_day

RandomForestRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=10, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=3, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=999, verbose=0, warm_start=False)

In [59]:
importances = model_igg_data_day.feature_importances_
std = np.std([tree.feature_importances_ for tree in model_igg_data_day.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")

feature_metadata = igg_data.feature_metadata
np.savetxt("feature_metadata.csv", feature_metadata, delimiter=",", fmt='%s')


tbars_importances = []
for f in range(X.shape[1]):
    tbars_importances += (indices[f], importances[indices[f]])
print(importances)

Feature ranking:
[6.45265730e-04 0.00000000e+00 0.00000000e+00 1.72082682e-04
 0.00000000e+00 1.80019245e-03 9.82159062e-03 0.00000000e+00
 2.13582490e-05 0.00000000e+00 1.11745432e-02 1.24337814e-02
 1.22925300e-03 7.68049486e-04 8.08731655e-04 1.10904183e-03
 0.00000000e+00 3.03856008e-04 0.00000000e+00 5.97195787e-04
 8.38746643e-05 1.42177441e-04 4.15274212e-05 2.39358792e-06
 2.44270121e-06 8.59004282e-05 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.83412211e-04 0.00000000e+00 3.01130193e-05
 0.00000000e+00 1.05445402e-04 4.51393729e-04 6.42689107e-04
 6.61347326e-04 7.12091627e-04 0.00000000e+00 8.31276506e-03
 1.13633677e-03 2.50612203e-03 5.50002990e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 7.95616916e-06 5.46332491e-05
 7.65827457e-04 3.73389770e-04 0.00000000e+00 0.00000000e+00
 0.00000000e+00 6.13322198e-05 0.00000000e+00 4.08563806e-02
 3.10026413e-05 9.47425559e-04 6.54472916e-04 3.69897381e-04
 0.00000000e+00 0.00000000e+00 1.83417219e-04 2.40206364e-03
 1.1000

In [60]:
for f in range(X.shape[1]):
    print(igg_data.feature_metadata['_feature_id'][f], importances[indices[f]])

46d3ba234416dee513f80752246a2066 0.05782281809487961
a781986c6888ab56b7925420d1e54abe 0.05623262050775994
2f2861bb61bd6c8cbd0d4163ec7bba53 0.046685458020198814
4325e118c9a634863c3a0977a0307615 0.040856380592501886
01d69d0ad44bd1fc99e0c4c2a0e8b313 0.0348825664753128
f42044fb1499aab36b0f3a0675039af2 0.027317830998024657
a45157ca6c33a380b00afed7bfb8b3d6 0.024658219057379874
6f0b872f7034f80cdf0f2fc4932697be 0.024597680972809843
15999689585a670d17275ebd210b39ff 0.022623820996108206
8c5c984f156f3e2c787f008ffcf69613 0.022214158867080978
8dbeabfbf721f4b9ee384b433f2ed270 0.01961151794030314
62f9ed0cc77aa2ad42da339e47297b6a 0.018347229251794977
e9fabaf81fba070afe35405e72b00992 0.018338215688208204
94e0875524c2cbde20c27b92c82a381b 0.017324084427720012
e9c9ea6a772243fadbcda9387790c0ca 0.0160277649638725
aeb2228a033d1764bca1b7eb507af64f 0.014984863286512906
889ec2416a0f378d5bd89b2857ba29f5 0.01393687412162948
9197cc47bf661baac6a81e6369744b9b 0.01346182458654729
74b459a38e8b6ae52e5586f7c94c4828 0.01